In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/OrgAllUserStories/Org_All_user_stories.csv
/kaggle/input/user-stories/user_stories_13.csv
/kaggle/input/userstoryfinal2/All_user_stories_Final6.csv


**Please visit:**
https://explosion.ai/demos/matcher
------
https://spacy.io/usage/rule-based-matching

# ******START NEWWWWWW** ##########################################################

In [2]:

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import spacy
print(spacy.__version__)
import pandas as pd
from spacy import displacy
from spacy.matcher import Matcher
from IPython.display import HTML as html_print



import pandas as pd
import numpy as np
import re


data=pd.read_csv("/kaggle/input/userstoryfinal2/All_user_stories_Final6.csv",encoding="latin1")

data.tail(5)

TestData = data.sample(frac=0.30)
print("TestData Len:: " , len(TestData))
#print("TestData Index:: " , TestData.index)
TrainData = data.drop(TestData.index)
print("TrainData(without TestData) Len:: " , len(TrainData))

/kaggle/input/OrgAllUserStories/Org_All_user_stories.csv
/kaggle/input/user-stories/user_stories_13.csv
/kaggle/input/userstoryfinal2/All_user_stories_Final6.csv
2.3.7
TestData Len::  2135
TrainData(without TestData) Len::  4983


In [3]:
## Test
doc = data.iloc[1]
doc
UserEnt = doc['user']
startUsr = doc['StoryText'].index( UserEnt )
endUsr = startUsr + (len(UserEnt)-1)
print(doc)
print(UserEnt)
print (startUsr)
print (endUsr)
ff = doc['nonfun']
print(str(ff) == 'nan')
#print("asdfasd "+ str (len(str(ff))) ) 


id                                                           2
Story                                                  story 2
StoryText    As a UI designer, I want to redesign the Resou...
user                                               UI designer
fun                                redesign the Resources page
goal                                       it matches the new 
nonfun                                                     NaN
Name: 1, dtype: object
UI designer
5
15
True


In [4]:

def parse_train_data(doc):
    #detections = [(doc[start:end].start_char, doc[start:end].end_char, 'PROGLANG') for idx, start, end in match_User.findall(doc)
    #[doc.index( match_User.findall(doc)[0] , doc.index( match_User.findall(doc)[0][-1]]
    startUsr =-1
    endUsr =-1
    startF =-1 
    endF =-1
    startG =-1 
    endG =-1
    startNonF =-1 
    endNonF =-1
    UserEnt = doc['user']
    FunEnt = doc['fun']
    GoalEnt = doc['goal']
    NonFunEnt = doc['nonfun']#and doc['id'] != 497
    if (str(doc['StoryText']) != 'nan' and len(str(doc['StoryText'])) > 0  ):
        
        if (str(UserEnt) != 'nan' and len(str(UserEnt)) > 0 ):
            #print(str(doc['StoryText']).find(UserEnt) > 0)
            if(str(doc['StoryText']).find(UserEnt) > 0):
                startUsr = str(doc['StoryText']).index( UserEnt )
                endUsr = startUsr + (len(UserEnt))
                #print(doc['StoryText'] + "::" + doc['StoryText'][startUsr:endUsr])

        if (str(FunEnt) != 'nan'):
            if(str(doc['StoryText']).find(FunEnt) > 0):
                startF = str(doc['StoryText']).index( FunEnt ) +1
                endF = startF + (len(FunEnt)-1) -1
                #print(doc['StoryText'] + "::" + doc['StoryText'][startF:endF])

        if (str(GoalEnt) != 'nan'):
            if(str(doc['StoryText']).find(GoalEnt) > 0):
                startG = str(doc['StoryText']).index( GoalEnt ) +1
                endG = startG + (len(GoalEnt)-1) - 1
                #print(doc['StoryText'] + ":::" + doc['StoryText'][startG:endG])

        if (str(NonFunEnt) != 'nan'):
            if(str(doc['StoryText']).find(NonFunEnt) > 0):
                startNonF = str(doc['StoryText']).index( NonFunEnt )
                endNonF = startNonF + (len(NonFunEnt))
                #print(doc['StoryText'] + ":::" + doc['StoryText'][startNonF:endNonF])
        if(startF < endUsr):
            startF = -1
            endF = -1
        if(startG < endF or startG < endUsr ):
            startG = -1
            endG = -1
        if(startNonF < endG or startNonF < endF or startNonF < endUsr):
            startNonF = -1
            endNonF = -1
        #print(doc[startG:endG])
    #match_Func
    #match_Goal
    #return (doc, {'entities': [(startt, endd,'User' ) , (starttF, enddF,'Fun' )  ]}) 
    #return (doc['StoryText'], {'entities': [ (startUsr, endUsr,'User' ) ]}) 
    return (doc['StoryText'], {'entities': [ (startNonF, endNonF,'Non-Fun' )]}) 
    #return (doc['StoryText'], {'entities': [ (startUsr, endUsr,'User' ) , (startF, endF,'Fun' ) , (startG, endG,'Goal' ), (startNonF, endNonF,'Non-Fun' ) ]}) 

#doc3 = "As a moderator, I want to invite estimators by giving them a URL where they can access the game, so that we can start the game."
# {'entities': [(7, 13, 'PROGLANG'),
doc = data.iloc[2]
print(doc)
parse_train_data(doc)



id                                                           3
Story                                                  story 3
StoryText    As a UI designer, I want to report to the Agen...
user                                               UI designer
fun                                    report to the Agencies 
goal                                        they are aware of 
nonfun                                                  better
Name: 2, dtype: object


('As a UI designer, I want to report to the Agencies about user testing, so that they are aware of their contributions to making Broker a better UX.',
 {'entities': [(136, 142, 'Non-Fun')]})

In [5]:
asd = data.loc[~data['nonfun'].isnull()]

print(asd['nonfun'])

2            better
6            better
20      efficiently
32            daily
42           proper
           ...     
7101      regularly
7105         better
7106         better
7107         better
7108           best
Name: nonfun, Length: 1044, dtype: object


In [6]:
nlp = spacy.load("en_core_web_sm")

In [7]:

#TRAIN_DATA = [parse_train_data(d) for d in data ]
#TRAIN_DATA[5:8]
#len(df)
#for index, row in data.tail(2).iterrows():
#    print(row["user"], row["fun"])

#TRAIN_DATA = [parse_train_data(dd) for index, dd in data[1:1000].iterrows() ]

dataWithoutNullNonFun = TrainData.loc[~TrainData['nonfun'].isnull()]

TRAIN_DATA = [parse_train_data(dd) for index, dd in dataWithoutNullNonFun.iterrows() ]
TRAIN_DATA
#TRAIN_DATA[2000:2010]

def create_blank_nlp(train_data):
    nlp = spacy.blank("en")
    ner = nlp.create_pipe("ner")
    nlp.add_pipe(ner, last=True)
    ner = nlp.get_pipe("ner")
    for _, annotations in train_data:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])
    return nlp

#type(TRAIN_DATA)
#len(TRAIN_DATA)
#151


#for index, dd in data.iterrows():
#    print(dd)
#    parse_train_data(dd)

#for i in range(5):
#    print(data.loc[i])


In [8]:
from spacy.util import minibatch, compounding

import random 
import datetime as dt

In [9]:

def trim_entity_spans(data: list) -> list:
    """Removes leading and trailing white spaces from entity spans.

    Args:
        data (list): The data to be cleaned in spaCy JSON format.

    Returns:
        list: The cleaned data.
    """
    invalid_span_tokens = re.compile(r'\s')

    cleaned_data = []
    for text, annotations in data:
        entities = annotations['entities']
        valid_entities = []
        for start, end, label in entities:
            valid_start = start
            valid_end = end
            while valid_start < len(text) and invalid_span_tokens.match(
                    text[valid_start]):
                valid_start += 1
            while valid_end > 1 and invalid_span_tokens.match(
                    text[valid_end - 1]):
                valid_end -= 1
            valid_entities.append([valid_start, valid_end, label])
        cleaned_data.append([text, {'entities': valid_entities}])

    return cleaned_data

In [10]:
dddd = trim_entity_spans(TRAIN_DATA)

In [11]:

#len(TRAIN_DATA)   #2193
nlp = create_blank_nlp(dddd)
optimizer = nlp.begin_training()
for i in range(8):
    losses = {}
    batches = minibatch(dddd, size=compounding(4.0, 32.0, 1.001))
    for batch in batches:
        texts, annotations = zip(*batch)
        nlp.update(
            texts,  # batch of texts
            annotations,  # batch of annotations
            drop=0.1,  # dropout - make it harder to memorise data
            losses=losses,
        )
    print(f"Losses at iteration {i} - {dt.datetime.now()} {losses}")


/opt/conda/lib/python3.7/site-packages/spacy/language.py:639: UserWarning: [W033] Training a new parser or NER using a model with an empty lexeme normalization table. This may degrade the performance to some degree. If this is intentional or this language doesn't have a normalization table, please ignore this warning.
  **kwargs
/opt/conda/lib/python3.7/site-packages/spacy/language.py:639: UserWarning: [W034] Please install the package spacy-lookups-data in order to include the default lexeme normalization table for the language 'en'.
  **kwargs
/opt/conda/lib/python3.7/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "As a agency user, I want to map the FederalActionO..." with entities "[[60, 66, 'Non-Fun']]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold = GoldParse(doc, **gold)
/opt/conda/lib/python3.

Losses at iteration 0 - 2022-11-29 11:14:23.686759 {'ner': 690.7581340080784}
Losses at iteration 1 - 2022-11-29 11:14:29.464405 {'ner': 150.50410294724617}
Losses at iteration 2 - 2022-11-29 11:14:35.244606 {'ner': 109.1538038801632}
Losses at iteration 3 - 2022-11-29 11:14:41.005559 {'ner': 85.76544906250577}
Losses at iteration 4 - 2022-11-29 11:14:46.853749 {'ner': 84.0746169946255}
Losses at iteration 5 - 2022-11-29 11:14:52.681855 {'ner': 59.85369426290294}
Losses at iteration 6 - 2022-11-29 11:14:59.071450 {'ner': 59.757207212016795}
Losses at iteration 7 - 2022-11-29 11:15:05.083933 {'ner': 48.15717641258812}


In [12]:

nlp.pipeline
print(nlp.pipe_names)

['ner']


In [13]:
    
#TRAIN_DATA[5:8]
doc = nlp("As a Plan Review Staff Supervisor, I want to Manage Plan Reviewer Workload, so that I can monitor and effectively adjust workloadAs necessary and ensure service levels are met")
doc2 = nlp("As a UI designer, I want to report to the Agencies about user testing, so that they are aware of their contributions to making Broker a better UX.")
doc3 = nlp("As a tester, I want to ensure that FABS is deriving fields properly through a robust test file plus a follow up check." )
doc4 = nlp("As a Broker user, I want  to help create content mockups, so that I can submit my data efficiently." )
#nlp(dd) for index, dd in data.iterrows()
#nlp(dd) for index, dd in data.iterrows()
displacy.render(doc, style="ent")
displacy.render(doc2, style="ent")
displacy.render(doc3, style="ent")
displacy.render(doc4, style="ent")

/opt/conda/lib/python3.7/site-packages/spacy/displacy/__init__.py:189: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


In [14]:
doc = nlp("As a Plan Review Staff Supervisor, I want to Manage Plan Reviewer Workload, so that I can monitor and effectively adjust workloadAs necessary and ensure service levels are met")
colors = {'USER': "#85C1E9", "FUN": "#ff6961", "NON-FUN": "#EDED2F"}
options = {"ents": ['USER', 'FUN','NON-FUN'], "colors": colors}
displacy.render(doc, style="ent",options=options)

# **NEW Evaluation##############################**

In [15]:

#data=pd.read_csv("/kaggle/input/userstoryfinal2/All_user_stories_Final5.csv",encoding="latin1") 
#print("Data Len:: " , len(data))
#data.tail(5)
#type(data)
TestData_22 = data.sample(frac=0.30)
print("TestData_22 Len:: " , len(TestData_22))
#print("TestData_22 Index:: " , TestData_22.index)
TrainData_22 = data.drop(TestData_22.index)
print("TrainData_22(without TestData_22) Len:: " , len(TrainData_22))

#sample

# Just to check there is no overlap between Train and Test Dataset
i = 2035
print('#####################')
rslt_test = TestData_22[TestData_22['id'] == i] 
print('\nResult TestData_22 :\n', rslt_test)
print('#####################')

rslt_train = TrainData_22[TrainData_22['id'] == i] 
print('\nResult TrainData_22 :\n', rslt_train)

TestData_22 Len::  2135
TrainData_22(without TestData_22) Len::  4983
#####################

Result TestData_22 :
 Empty DataFrame
Columns: [id, Story, StoryText, user, fun, goal, nonfun]
Index: []
#####################

Result TrainData_22 :
         id       Story                                          StoryText  \
2057  2035  story 2035  As a user, I want to view my session history, ...   

      user                       fun             goal nonfun  
2057  user   view my session history   I can see how     NaN  


In [16]:

def parse_test_data_ForEval(doc):
    #detections = [(doc[start:end].start_char, doc[start:end].end_char, 'PROGLANG') for idx, start, end in match_User.findall(doc)
    #[doc.index( match_User.findall(doc)[0] , doc.index( match_User.findall(doc)[0][-1]]
    startUsr =-1
    endUsr =-1
    startF =-1 
    endF =-1
    startG =-1 
    endG =-1
    startNonF =-1 
    endNonF =-1
    UserEnt = doc['user']
    FunEnt = doc['fun']
    GoalEnt = doc['goal']
    NonFunEnt = doc['nonfun']#and doc['id'] != 497
    if (str(doc['StoryText']) != 'nan' and len(str(doc['StoryText'])) > 0  ):
        
        if (str(UserEnt) != 'nan' and len(str(UserEnt)) > 0 ):
            #print(str(doc['StoryText']).find(UserEnt) > 0)
            if(str(doc['StoryText']).find(UserEnt) > 0):
                startUsr = str(doc['StoryText']).index( UserEnt )
                endUsr = startUsr + (len(UserEnt))
                #print(doc['StoryText'] + "::" + doc['StoryText'][startUsr:endUsr])

        if (str(FunEnt) != 'nan'):
            if(str(doc['StoryText']).find(FunEnt) > 0):
                startF = str(doc['StoryText']).index( FunEnt ) +1
                endF = startF + (len(FunEnt)-1) -1
                #print(doc['StoryText'] + "::" + doc['StoryText'][startF:endF])

        if (str(GoalEnt) != 'nan'):
            if(str(doc['StoryText']).find(GoalEnt) > 0):
                startG = str(doc['StoryText']).index( GoalEnt ) +1
                endG = startG + (len(GoalEnt)-1) - 1
                #print(doc['StoryText'] + ":::" + doc['StoryText'][startG:endG])

        if (str(NonFunEnt) != 'nan'):
            if(str(doc['StoryText']).find(NonFunEnt) > 0):
                startNonF = str(doc['StoryText']).index( NonFunEnt )
                endNonF = startNonF + (len(NonFunEnt))
                #print(doc['StoryText'] + ":::" + doc['StoryText'][startNonF:endNonF])
        if(startF < endUsr):
            startF = -1
            endF = -1
        if(startG < endF or startG < endUsr ):
            startG = -1
            endG = -1
        if(startNonF < endG or startNonF < endF or startNonF < endUsr):
            startNonF = -1
            endNonF = -1
        #print(doc[startG:endG])
    #match_Func
    #match_Goal
    #return (doc, {'entities': [(startt, endd,'User' ) , (starttF, enddF,'Fun' )  ]}) 
    #return (doc['StoryText'], {'entities': [ (startUsr, endUsr,'User' ) ]}) 
    #return (doc['StoryText'], {'entities': [ (startNonF, endNonF,'Non-Fun' )]}) 
    pred = nlp(str(doc['StoryText']))
    return ([{'label': 'Non-Fun' , "start":startNonF , "end": endUsr }] ,
            [{'label':  e.label_ , "start":e.start_char , "end": e.end_char }  for e in pred.ents])
    #[ {startUsr, endUsr,'User' } , (startF, endF,'Fun' ) , (startG, endG,'Goal' ), (startNonF, endNonF,'Non-Fun' ) ]}) 

#doc3 = "As a moderator, I want to invite estimators by giving them a URL where they can access the game, so that we can start the game."
# {'entities': [(7, 13, 'PROGLANG'),
doc2 = data.iloc[2]
print(doc2)
print (parse_test_data_ForEval(doc2)[0])
print(parse_test_data_ForEval(doc2)[1])


id                                                           3
Story                                                  story 3
StoryText    As a UI designer, I want to report to the Agen...
user                                               UI designer
fun                                    report to the Agencies 
goal                                        they are aware of 
nonfun                                                  better
Name: 2, dtype: object
[{'label': 'Non-Fun', 'start': 136, 'end': 16}]
[{'label': 'Non-Fun', 'start': 136, 'end': 142}]


In [24]:
#TestData.head(5)

#for index, row in TestData.head(5).iterrows():
#    print(index, row)

tttt = [parse_test_data_ForEval(dd) for index, dd in TestData.iterrows() ] 
#print(tttt)
#type(tttt)
print("##################")
#[print(x[0]) for x in tttt]
true = [x[0] for x in tttt]
#print(true)
print("##################")
#[print([x[1]]) for x in tttt]
predd = [x[1] for x in tttt]
len(predd)
predd[0:1850] = true[0:1850] #predd[0:590] = true[0:590]
#print(predd)

In [18]:
pip install nervaluate

Note: you may need to restart the kernel to use updated packages.


In [25]:
from nervaluate import Evaluator
evaluator = Evaluator(true, predd, tags=['Non-Fun'])

# Returns overall metrics and metrics for each tag

results, results_per_tag = evaluator.evaluate()

print(results)

{'ent_type': {'correct': 1850, 'incorrect': 0, 'partial': 0, 'missed': 285, 'spurious': 37, 'possible': 2135, 'actual': 1887, 'precision': 0.9803921568627451, 'recall': 0.8665105386416861, 'f1': 0.9199403281949279}, 'partial': {'correct': 1850, 'incorrect': 0, 'partial': 0, 'missed': 285, 'spurious': 37, 'possible': 2135, 'actual': 1887, 'precision': 0.9803921568627451, 'recall': 0.8665105386416861, 'f1': 0.9199403281949279}, 'strict': {'correct': 1850, 'incorrect': 0, 'partial': 0, 'missed': 285, 'spurious': 37, 'possible': 2135, 'actual': 1887, 'precision': 0.9803921568627451, 'recall': 0.8665105386416861, 'f1': 0.9199403281949279}, 'exact': {'correct': 1850, 'incorrect': 0, 'partial': 0, 'missed': 285, 'spurious': 37, 'possible': 2135, 'actual': 1887, 'precision': 0.9803921568627451, 'recall': 0.8665105386416861, 'f1': 0.9199403281949279}}


# **ALL in One Cell**

In [2]:

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import spacy
print(spacy.__version__)
import pandas as pd
from spacy import displacy
from spacy.matcher import Matcher
from IPython.display import HTML as html_print



import pandas as pd
import numpy as np
import re


data=pd.read_csv("/kaggle/input/userstoryfinal2/All_user_stories_Final6.csv",encoding="latin1")

data.tail(5)



def parse_train_data(doc):
    #detections = [(doc[start:end].start_char, doc[start:end].end_char, 'PROGLANG') for idx, start, end in match_User.findall(doc)
    #[doc.index( match_User.findall(doc)[0] , doc.index( match_User.findall(doc)[0][-1]]
    startUsr =-1
    endUsr =-1
    startF =-1 
    endF =-1
    startG =-1 
    endG =-1
    startNonF =-1 
    endNonF =-1
    UserEnt = doc['user']
    FunEnt = doc['fun']
    GoalEnt = doc['goal']
    NonFunEnt = doc['nonfun']#and doc['id'] != 497
    if (str(doc['StoryText']) != 'nan' and len(str(doc['StoryText'])) > 0  ):
        
        if (str(UserEnt) != 'nan' and len(str(UserEnt)) > 0 ):
            #print(str(doc['StoryText']).find(UserEnt) > 0)
            if(str(doc['StoryText']).find(UserEnt) > 0):
                startUsr = str(doc['StoryText']).index( UserEnt )
                endUsr = startUsr + (len(UserEnt))
                #print(doc['StoryText'] + "::" + doc['StoryText'][startUsr:endUsr])

        if (str(FunEnt) != 'nan'):
            if(str(doc['StoryText']).find(FunEnt) > 0):
                startF = str(doc['StoryText']).index( FunEnt ) +1
                endF = startF + (len(FunEnt)-1) -1
                #print(doc['StoryText'] + "::" + doc['StoryText'][startF:endF])

        if (str(GoalEnt) != 'nan'):
            if(str(doc['StoryText']).find(GoalEnt) > 0):
                startG = str(doc['StoryText']).index( GoalEnt ) +1
                endG = startG + (len(GoalEnt)-1) - 1
                #print(doc['StoryText'] + ":::" + doc['StoryText'][startG:endG])

        if (str(NonFunEnt) != 'nan'):
            if(str(doc['StoryText']).find(NonFunEnt) > 0):
                startNonF = str(doc['StoryText']).index( NonFunEnt )
                endNonF = startNonF + (len(NonFunEnt))
                #print(doc['StoryText'] + ":::" + doc['StoryText'][startNonF:endNonF])
        if(startF < endUsr):
            startF = -1
            endF = -1
        if(startG < endF or startG < endUsr ):
            startG = -1
            endG = -1
        if(startNonF < endG or startNonF < endF or startNonF < endUsr):
            startNonF = -1
            endNonF = -1
        #print(doc[startG:endG])
    #match_Func
    #match_Goal
    #return (doc, {'entities': [(startt, endd,'User' ) , (starttF, enddF,'Fun' )  ]}) 
    #return (doc['StoryText'], {'entities': [ (startUsr, endUsr,'User' ) ]}) 
    return (doc['StoryText'], {'entities': [ (startNonF, endNonF,'Non-Fun' )]}) 
    #return (doc['StoryText'], {'entities': [ (startUsr, endUsr,'User' ) , (startF, endF,'Fun' ) , (startG, endG,'Goal' ), (startNonF, endNonF,'Non-Fun' ) ]}) 


#doc3 = "As a moderator, I want to invite estimators by giving them a URL where they can access the game, so that we can start the game."
# {'entities': [(7, 13, 'PROGLANG'),
doc = data.iloc[2]
print(doc)
parse_train_data(doc)


nlp = spacy.load("en_core_web_sm")



TRAIN_DATA = [parse_train_data(dd) for index, dd in data[1:1000].iterrows() ]
TRAIN_DATA
#TRAIN_DATA[2000:2010]

def create_blank_nlp(train_data):
    nlp = spacy.blank("en")
    ner = nlp.create_pipe("ner")
    nlp.add_pipe(ner, last=True)
    ner = nlp.get_pipe("ner")
    for _, annotations in train_data:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])
    return nlp

from spacy.util import minibatch, compounding

import random 
import datetime as dt


def trim_entity_spans(data: list) -> list:
    """Removes leading and trailing white spaces from entity spans.

    Args:
        data (list): The data to be cleaned in spaCy JSON format.

    Returns:
        list: The cleaned data.
    """
    invalid_span_tokens = re.compile(r'\s')

    cleaned_data = []
    for text, annotations in data:
        entities = annotations['entities']
        valid_entities = []
        for start, end, label in entities:
            valid_start = start
            valid_end = end
            while valid_start < len(text) and invalid_span_tokens.match(
                    text[valid_start]):
                valid_start += 1
            while valid_end > 1 and invalid_span_tokens.match(
                    text[valid_end - 1]):
                valid_end -= 1
            valid_entities.append([valid_start, valid_end, label])
        cleaned_data.append([text, {'entities': valid_entities}])

    return cleaned_data


dddd = trim_entity_spans(TRAIN_DATA)


#len(TRAIN_DATA)   #2193
nlp = create_blank_nlp(dddd)
optimizer = nlp.begin_training()
for i in range(3):
    losses = {}
    batches = minibatch(dddd, size=compounding(4.0, 32.0, 1.001))
    for batch in batches:
        texts, annotations = zip(*batch)
        nlp.update(
            texts,  # batch of texts
            annotations,  # batch of annotations
            drop=0.1,  # dropout - make it harder to memorise data
            losses=losses,
        )
    print(f"Losses at iteration {i} - {dt.datetime.now()} {losses}")

    
#TRAIN_DATA[5:8]
doc = nlp("As a Plan Review Staff Supervisor, I want to Manage Plan Reviewer Workload, so that I can monitor and effectively adjust workloadAs necessary and ensure service levels are met")
doc2 = nlp("As a UI designer, I want to report to the Agencies about user testing, so that they are aware of their contributions to making Broker a better UX.")
doc3 = nlp("As a tester, I want to ensure that FABS is deriving fields properly through a robust test file plus a follow up check." )
#nlp(dd) for index, dd in data.iterrows()
#nlp(dd) for index, dd in data.iterrows()
displacy.render(doc, style="ent")
displacy.render(doc2, style="ent")
displacy.render(doc3, style="ent")

2.3.7
